In [ ]:
!pip install -U langchain-ollama langchain-openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/review_dataset

__MACOSX	       test_file.csv	   w_review_train.parquet
sample_submission.csv  w_review_train.csv


In [ ]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [ ]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [ ]:
df.shape

(40000, 2)

In [ ]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [ ]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [ ]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [ ]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [ ]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [ ]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

CPU times: user 1.9 s, sys: 63.7 ms, total: 1.97 s
Wall time: 2 s


In [ ]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [ ]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.24 s, sys: 75.6 ms, total: 2.32 s
Wall time: 2.42 s


In [ ]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [ ]:
coffee_reviews_df.shape

(4981, 2)

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [ ]:
!ollama pull gemma3:270m

In [ ]:
!ollama list

NAME           ID              SIZE      MODIFIED       
gemma3:270m    e7d36fb2c3b3    291 MB    46 seconds ago    


In [ ]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ ยินดีที่ได้รู้จักนะคะ 😊




In [ ]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-11-10T13:40:50.444644702Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [ ]:
! pip install -U langchain langchain-core langchain-community langchain-ollama

In [ ]:
! pip show langchain langchain-core langchain-community

Name: langchain
Version: 1.0.5
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
---
Name: langchain-core
Version: 1.0.4
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, pyyaml, tenacity, typing-extensions
Required-by: langchain, langchain-classic, langchain-community, langchain-ollama, langchain-openai, langchain-text-splitters, langgraph, langgraph-checkpoint, langgraph-prebuilt


In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
llm = OllamaLLM(model="gemma3:270m")

In [ ]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊



In [ ]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [ ]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [ ]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [ ]:
chain = prompt | llm

In [ ]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [ ]:
response

'```python\ndef find_sum(numbers):\n  """\n  หาผลรวมของตัวเลข 1 ถึง 10\n  """\n  sum = 0\n  for number in numbers:\n    sum += number\n  return sum\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = find_sum(numbers)\nprint(result)\n```\n\n**คำอธิบายโค้ด:**\n\n1.  **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ที่จะทำการค้นหาผลรวมของตัวเลข\n2.  **`sum = 0`**:  เริ่มต้นค่าคง alteที่เก็บผลรวมของตัวเลข\n3.  **`for number in numbers:`**:  วนรอบผ่านตัวเลขทั้งหมดใน `numbers`\n4.  **`sum += number`**:  เพิ่มค่าของ `number` เข้าไปในค่าคง alte `sum`\n5.  **`return sum`**:  คืนค่าค่าคง alte `sum`\n6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  กำหนดตัวแปร `numbers` ที่จะเก็บตัวเลขที่ต้องการหาผลรวม\n7.  **`result = find_sum(numbers)`**:  เรียกฟังก์ชัน `find_sum` เพื่อทำการค้นหาผลรวม\n8.  **`print(result)`**:  แสดงผลลัพธ์ที่ได้จากฟังก์ชัน `find_sum`\n9.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  กำหนดตัวแปร `numbers` ที่จะเก็บตัวเลขที่ต้องการหา

In [ ]:
from IPython.display import display, Markdown

In [ ]:
display(Markdown(response))

```python
def find_sum(numbers):
  """
  หาผลรวมของตัวเลข 1 ถึง 10
  """
  sum = 0
  for number in numbers:
    sum += number
  return sum

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum(numbers)
print(result)
```

**คำอธิบายโค้ด:**

1.  **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ที่จะทำการค้นหาผลรวมของตัวเลข
2.  **`sum = 0`**:  เริ่มต้นค่าคง alteที่เก็บผลรวมของตัวเลข
3.  **`for number in numbers:`**:  วนรอบผ่านตัวเลขทั้งหมดใน `numbers`
4.  **`sum += number`**:  เพิ่มค่าของ `number` เข้าไปในค่าคง alte `sum`
5.  **`return sum`**:  คืนค่าค่าคง alte `sum`
6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  กำหนดตัวแปร `numbers` ที่จะเก็บตัวเลขที่ต้องการหาผลรวม
7.  **`result = find_sum(numbers)`**:  เรียกฟังก์ชัน `find_sum` เพื่อทำการค้นหาผลรวม
8.  **`print(result)`**:  แสดงผลลัพธ์ที่ได้จากฟังก์ชัน `find_sum`
9.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  กำหนดตัวแปร `numbers` ที่จะเก็บตัวเลขที่ต้องการหาผลรวม
10. **`result = find_sum(numbers)`**:  เรียกฟังก์ชัน `find_sum` เพื่อทำการค้นหาผลรวม
11. **`print(result)`**:  แสดงผลลัพธ์ที่ได้จากฟังก์ชัน `find_sum`

**ข้อดีของโค้ดนี้:**

*   **ใช้งานง่าย:**  โค้ดนี้เป็นภาษา Python ที่เข้าใจง่ายและสามารถนำไปใช้งานได้โดยไม่ต้องเขียนโค้ดซ้ำ
*   **มีประสิทธิภาพ:**  ฟังก์ชัน `find_sum` ทำงานได้อย่างรวดเร็ว
*   **การจัดการค่าคง alte:**  ฟังก์ชัน `sum` เก็บค่าคง alte ในตัว
*   **การทำงานได้อย่างมีประสิทธิภาพ:**  ฟังก์ชัน `find_sum` ทำงานได้อย่างต่อเนื่อง

**การปรับปรุง (ถ้าต้องการ):**

ถ้าต้องการเพิ่มความสามารถในการจัดการกับตัวแปรที่แตกต่างกันได้ (เช่น  `numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`)  คุณสามารถเพิ่ม `numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`  ใน `find_sum` เพื่อให้ `sum` มีค่าคง alte  (โดยทั่วไปจะใช้ค่าคง alte  `0`  หรือ `0.0`)

```python
def find_sum(numbers):
  """
  หาผลรวมของตัวเลข 1 ถึง 10
  """
  sum = 0
  for number in numbers:
    sum += number
  return sum

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum(numbers)
print(result)
```

**ข้อดีของโค้ดนี้:**

*   **จัดการค่าคง alte:**  ฟังก์ชัน `sum` เก็บค่าคง alte ในตัว ทำให้โค้ดไม่ซับซ้อนเหมือนฟังก์ชัน `for`
*   **ใช้งานง่าย:**  โค้ดนี้เป็นภาษา Python ที่ใช้งานง่าย

**ข้อควรระวัง:**

*   **การจัดการตัวแปรที่แตกต่างกัน:**  ฟังก์ชัน `sum`  สามารถจัดการกับตัวแปรที่แตกต่างกันได้ (เช่น  `numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`  และ  `sum = 0`  )  คุณจะต้องใช้ `numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`  ใน `find_sum`
*   **การจัดการตัวแปรที่เปลี่ยนแปลง:**  ฟังก์ชัน `sum`  สามารถจัดการกับตัวแปรที่เปลี่ยนแปลงได้ (เช่น  `numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`  และ  `sum = 0`  )  คุณจะต้องใช้ `numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`  ใน `find_sum`



In [ ]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [ ]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [ ]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [ ]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [ ]:
parser_chain = StrOutputParser()

In [ ]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [ ]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [ ]:
result

'วิธิีทำจากผลลัทธ์ มีความซับซ้อนและมีหลายแง่มุม แต่สามารถอธิบายได้ดังนี้ครับ'

In [ ]:
import os
from langchain_openai import ChatOpenAI

In [ ]:
OPENAI_API_KEY = "xxx"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
llm2 = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
chain1 = prompt1 | llm2
chain2 = prompt2 | llm2
chain3 = prompt3 | llm2
chain4 = prompt4 | llm2

full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [ ]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

result

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: xxx. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
formatted_result = result.replace('\\[', '$$') \
                         .replace('\\]', '$$') \
                         .replace('\\(', '$') \
                         .replace('\\)', '$')

In [ ]:
display(Markdown(formatted_result))

วิธิีทำจากผลลัทธ์ มีความซับซ้อนและมีหลายแง่มุม แต่สามารถอธิบายได้ดังนี้ครับ

In [ ]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [ ]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [ ]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [ ]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [ ]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [ ]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [ ]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 111 ms, sys: 6.36 ms, total: 117 ms
Wall time: 9.58 s


In [ ]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{"keywords": ["ร้านอาหาร", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มัน", "ทอดมัน", "หัวปลีก", "อร่อย", "เบิ้ล", "ขนมไทย", "ขนมหวาน", "ฟรี", "ขนมไทย", "ลาบไข่ต้ม", "ไข่อร่อย", "เห้ย", "ราคา", "เกินไป", "รับไม่ไหว", "ว", "ไม่ไปซ้ำ", "แพง", "ราคาก็", "ไม่ไหว", "ขนมหวานให้กินฟรี", "ขนมไทย", "ลาบไข่ต้ม", "ไข่มันคาว", "อะ", "ไม่ประทับใจ", "ไม่พอ", "ไม่น่าทาน", "ไม่น่ากิน", "ไม่น่ากิน"]}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "แก้วนึง",
    "ร้อน",
    "มาวางอยู่ข้างหน้า",
    "ซากอารยธรรม",
    "เมพพพ",
    "ซากอารยธรรม",
    "อาหาร",
    "กาแฟ",
    "ราคาไม่แพง",
    "กินบ่อยก็จุกได้",
    "ทานบ่อยๆ",
    "ร้านวาวี",
    "อารีย์",
    "สวย",
    "วิวรอกาแฟ",
    "สวนบ้านเรา",
    "สีเขียวเล็กๆ",
    "สีเขียวชอุ่ม",
 

In [ ]:
keyword_extract = keyword_prompt | llm2 | StrOutputParser()

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: xxx. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

In [ ]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [ ]:
!ollama list

NAME                                  ID              SIZE      MODIFIED       
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    15 seconds ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    10 minutes ago    


In [ ]:
llm3 = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b")

In [ ]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 120 ms, sys: 9.63 ms, total: 130 ms
Wall time: 16.3 s


In [ ]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะช่วยคุณวิเคราะห์และดึงคำสำคัญจากรีวิวนี้ตามขั้นตอนที่คุณต้องการ

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้เป็นการรีวิวร้านอาหารที่มีขนาดใหญ่ และมีข้อดีข้อเสียหลายอย่าง เช่น อาหารบางอย่างอร่อย (ทอดมันหัวปลี, พะแนงห่อไข่) แต่มีราคาแพงเกินไป และมีข้อเสียอื่นๆ เช่น น้ำผึ้งที่ไม่ได้มีอะไรพิเศษ

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

*   ขนาดร้าน
*   ราคา
*   รสชาติอาหาร (ทอดมันหัวปลี, พะแนงห่อไข่)
*   น้ำผึ้ง

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

จากหัวข้อหลักที่กล่าวถึง ผมเลือกคำเหล่านี้เป็น keywords:

1.  ราคา
2.  อาหาร
3.  ขนาด

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ราคา", "อาหาร", "ขนาด"]
}
```

หวังว่าคำตอบนี้จะเป็นประโยชน์นะครับ หากคุณมีคำถามเพิ่มเติมหรือต้องการให้ผมช่วยวิเคราะห์อะไรอีก สามารถบอกได้เลยครับ

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [ ]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [ ]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 169 ms, sys: 14.6 ms, total: 183 ms
Wall time: 13.6 s


In [ ]:
results[0]['text']['keywords']

['ราคา', 'รสชาติ', 'ขนาด']

In [ ]:
results[1]['text']['keywords']

['วาวี', 'กาแฟ', 'อารีย์']

In [ ]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 159 ms, sys: 8.1 ms, total: 168 ms
Wall time: 12.6 s


In [ ]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'อาหาร', 'ขนาด'], ['กาแฟ', 'บรรยากาศ', 'รสชาติ'])

In [ ]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [ ]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [ ]:
inputs = [{"review": r} for r in sample_reviews]

In [ ]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 148 ms, sys: 13.4 ms, total: 162 ms
Wall time: 12.4 s


In [ ]:
results

[{'keywords': ['ราคา', 'อาหาร', 'ขนาด']},
 {'keywords': ['วาวี', 'ลาเต้', 'บรรยากาศ']}]

# Assignment_660710749

In [ ]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(100).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [ ]:
from tqdm import tqdm

In [ ]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 13/13 [11:08<00:00, 51.43s/it]


In [ ]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

['ราคา', 'อาหาร', 'ขนาด']
['กาแฟ', 'บรรยากาศ', 'รสชาติ']
['โดนัท', 'ความหวาน', 'ความร้อน']
['วาซาบิ', 'ปลาซาบะ', 'คุ้มค่า']
['Original Glazed', 'คิว', 'โปรโมชั่น']
['นรสิงห์', 'วังพญาไท', 'ประวัติศาสตร์']
['อาหารทะเล', 'ราคา', 'ที่จอดรถ']
['Amazon', 'เค้ก', 'พนักงาน']
['Shibuya Honey Toast', 'Horlicks Latte', 'บรรยากาศร้าน']
['วานิลลาคาเฟ่', 'คาเฟ่', 'คาวบอนาร่า']


In [ ]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [ ]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)

In [ ]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [ ]:
context_extract = context_prompt | llm3 | context_parser

In [ ]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [ ]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [ ]:
keywords = results_1000[0]['text']['keywords']

In [ ]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords']
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 100/100 [00:00<00:00, 62091.84it/s]


In [ ]:
len(context_inputs)

300

In [ ]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 100/100 [08:42<00:00,  5.22s/it]


In [ ]:
results_1000[:3]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': ['ราคา', 'อาหาร', 'ขนาด']},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'อาหารที่นี่มีหลายอย่าง เช่น ลาบไข่ต้ม ไข่มันคาว และพะแนงห่อไข่',
   'ร้านอาหารใหญ่มากกกกกกก']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียว

In [ ]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 100/100 [00:00<00:00, 263295.92it/s]


In [ ]:
len(sentiment_inputs)

300

In [ ]:
sentiment_inputs[:10]

[{'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'อาหารที่นี่มีหลายอย่าง เช่น ลาบไข่ต้ม ไข่มันคาว และพะแนงห่อไข่'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก'},
 {'message': 'กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด'},
 {'message': 'บรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี อย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ เขียวชอุ่ม'},
 {'message': 'รสชาติและกลิ่นมันไม่ธรรมดาเลย หอมๆ ถือว่าเป็นมวยรุ่นเดียวกะ Starbuck ได้'},
 {'message': 'เนื้อแป้งนิ่มมากถ้าเทียบกับยี่ห้อใกล้เคียงที่เป็นแบบเดียวกัน ทำให้เวลาจับเบาๆน้ำตาลก็จะร่วงออกมาง่ายไปหน่อย'},
 {'message': 'แป้งนิ่มเวลากินก็จะทำให้รับรสหวานจากแป้งได้ง่ายกว่า'},
 {'message': 'สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บังเอิญมีรุ่นน้องที่ไม่ได้เจอกันนานชวนไปเป็นเพื่อน แล้วเค้าให้ซื้อได้คนละกล่อง เลยเข้าไปต่อคิวซื้อให้น้องเพิ่มอีกกล่องนึง'},
 {'message': 'วาซาบิอันเป็นหนึ่งในร้านที่แม่บ้านญี่ปุ่นนิยมทานกัน สาเหตุก็เพราะคุณภาพอาหารและราคาที่ยากจะหาใครสู้ค่ะ'}]

In [ ]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 100/100 [04:27<00:00,  2.68s/it]


In [ ]:
results_1000[0:3]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': ['ราคา', 'อาหาร', 'ขนาด']},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'อาหารที่นี่มีหลายอย่าง เช่น ลาบไข่ต้ม ไข่มันคาว และพะแนงห่อไข่',
   'ร้านอาหารใหญ่มากกกกกกก'],
  'sentiment': ['negative', 'neutral', 'neutral']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ใ

In [114]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [115]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')